<a href="https://colab.research.google.com/github/umi-na/machine_learning/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 딥러닝 VS 머신러닝
### 머신러닝
입력 데이터 피쳐벡터 수동 추출(Manual Feature Extraction)
</br>-> 머신러닝(분류; Classification)
</br>-> 결과 출력</br>

### 딥러닝
입력 데이터
</br>-> 모델이 피처벡터 추출 + 분류(Feature Extraction + Classification)
</br>-> 결과 출력
</br>시간이 오래 걸리고 과적합에 빠질 수 있는 가능성이 있지만, 추출을 직접 해줄 필요가 없기 때문에 사용성이 좋다

### CNN
- 2차원 영상에서 유효한 특징(feature)를 찾아내는 역할
- 유용한 필터 마스크가 학습에 의해 결정됨

</br>
LeNet-5
</br>
7개 레이어: Conv-Pool-Conv-Pool-FC-FC-FC

In [ ]:
pip install tensorflow==1.13.1
# tensorflow는 기본적으로 2가 깔리기 때문에 1이라고 지정, 아래 코드는 
# tensorflow-gpu버전도 있는데 CUDA 등을 사용하면 가능하다

     |████████████████████████████████| 92.6MB 31kB/s 
     |████████████████████████████████| 3.2MB 44.4MB/s 
     |████████████████████████████████| 368kB 49.6MB/s 
     |████████████████████████████████| 51kB 9.3MB/s 
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.13.1 which is incompatible.
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import graph_util
from tensorflow.python.platform import gfile

tf.logging.set_verbosity(tf.logging.ERROR)

# mnist = input_data.read_data_sets('./MNIST_data/', one_hot=True) # 이 문장이 실행되면 MNIST_data라는 폴더가 생기고 그 밑에 mnist관련된 데이터를 다운로드 받을 수 있다.
# 근데 content의 가상파일에 깔려서 볼수없음, 경로 새로 저장
mnist = input_data.read_data_sets('/content/drive/MyDrive/Colab Notebooks/deep_learning/MNIST_data/', one_hot=True)

#
# hyper parameters
#
learning_rate = 0.001 # gradient decent라는 알고리즘에서 학습을 어느정도 빠르기로 결정할 것인지, learning rate에 대한 내용은 deep_learnin파일 참고
training_epochs = 20 # 전체 학습 횟수
batch_size = 100

#
# Model configuration
#
X = tf.placeholder(tf.float32, [None, 28, 28, 1], name='data') # mnist data 파일을 받아오면 1차원 데이터로 불러오기 때문에 2차원으로 바꿈
Y = tf.placeholder(tf.float32, [None, 10]) # output으로 나오는 y는 10개이다

conv1 = tf.layers.conv2d(X, 10, [3, 3], padding='same', activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(conv1, [2, 2], strides=2, padding='same')

conv2 = tf.layers.conv2d(pool1, 20, [3, 3], padding='same', activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(conv2, [2, 2], strides=2, padding='same')

fc1 = tf.contrib.layers.flatten(pool2)
fc2 = tf.layers.dense(fc1, 200, activation=tf.nn.relu)
logits = tf.layers.dense(fc2, 10, activation=None)
output = tf.nn.softmax(logits, name='prob')

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

#
# Training
#
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples / batch_size)

print('Start learning!')
for epoch in range(training_epochs): # 20번 반복
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val

    print('Epoch: {0}, Avg. Cost = {1:.4f}'.format(epoch + 1, total_cost/total_batch)) # 한 번 학습할 때마다 에러율; 코스트가 얼마나 되는지 알려줌

print('Learning finished!')

# Test the results
is_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
acc = tf.reduce_mean(tf.cast(is_correct, tf.float32))
accuracy = sess.run(acc, feed_dict={
                    X: mnist.test.images.reshape(-1, 28, 28, 1), Y: mnist.test.labels}) # mnist의 training이 아니라 test인 이미지를 가져와서 맞는지 틀리는지
print('Test Accuracy:', accuracy)

# Freeze variables and save pb file
output_graph_def = graph_util.convert_variables_to_constants(sess, sess.graph_def, ['prob']) # weight가 원래는 variable; 변수지만 constant; 상수로 변경
# with gfile.FastGFile('./mnist_cnn.pb', 'wb') as f: # 상수로 변경한 weight를 mnist_cnn.pb에 저장인데 얘도 content에 깔리네...
with gfile.FastGFile('/content/drive/MyDrive/Colab Notebooks/deep_learning/mnist_cnn.pb', 'wb') as f:
    f.write(output_graph_def.SerializeToString())
# 과적합을 방지할 수 있는 dropout이 있지만 굳이 사용하진 않았다

print('mnist_cnn.pb file is created successfully!!')

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /content/drive/MyDrive/Colab Notebooks/deep_learning/MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /content/drive/MyDrive/Colab Notebooks/deep_learning/MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /content/drive/MyDrive/Colab Notebooks/deep_learning/MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /content/drive/MyDrive/Colab Notebooks/deep_learning/MNIST_data/t10k-labels-idx1-ubyte.gz
Start learning!
Epoch: 1, Avg. Cost = 0.2572
Epoch: 2, Avg. Cost = 0.0695
Epoch: 3, Avg. Cost = 0.0485
Epoch: 4, Avg. Cost = 0.0363
Epoch: 5, Avg. Cost = 0.0306
Epoch: 6, Avg. Cost = 0.0255
Epoch: 7, Avg. Cost = 0.0190
Epoch: 8, Avg. Cost = 0.0163
Epoch: 9, Avg. Cost = 0.0144
Epoch: 10, Avg. Cost = 0.0111
Epoch: 11, Avg. Cost = 0.0086
